In [ ]:
from datetime import timedelta,datetime
import pandas as pd
import numpy as np

In [ ]:
class ForecastIntraday():


    def forecastDateRange(self, df, Ntest):
        last = df.dropna()
        last_timestamp = last.index[-1] + timedelta(hours=24)
        first_timestamp = last_timestamp - timedelta(minutes=(Ntest-1)*60)
        forecast_date_range = pd.date_range(start=first_timestamp, end=last_timestamp, freq='60min')

        return forecast_date_range

    def getLastRealized(self,df,T):
        df_last_day = df.dropna()[-T:]

        return df_last_day

    def getCurrentForecast(self,df,best_model,scaler,T,Ntest,featureNumber,stockname):
        df_last_day=self.getLastRealized(df,T)
        print(df_last_day)
        forecast_range=self.forecastDateRange(df,Ntest)
        df_last_day = pd.DataFrame(scaler.transform(df_last_day.values))
        df_last_day.columns = ['price']
        df_last_day.set_index(forecast_range, inplace=True)

        df_last_day['Seconds'] = df_last_day.index.map(pd.Timestamp.timestamp)
        day = 60 * 60 * 24
        year = 365.2425 * day

        df_last_day['Day sin'] = np.sin(df_last_day['Seconds'] * (2 * np.pi / day))
        df_last_day['Day cos'] = np.cos(df_last_day['Seconds'] * (2 * np.pi / day))
        df_last_day['Year sin'] = np.sin(df_last_day['Seconds'] * (2 * np.pi / year))
        df_last_day['Year cos'] = np.cos(df_last_day['Seconds'] * (2 * np.pi / year))
        df_last_day.drop(columns={'Seconds'}, inplace=True)


        X_last_day = df_last_day.to_numpy()
        X_last_day = X_last_day.reshape(-1, T, featureNumber)

        Plast_day = best_model.predict(X_last_day)
        Plast_day = scaler.inverse_transform(Plast_day)

        forecast_df = pd.DataFrame({'timestamp_cet': forecast_range, 'forecast': Plast_day.flatten()})
        forecast_df['update_time']=datetime.now().replace(second=0, microsecond=0)
        forecast_df['forecast_label']=stockname
        forecast_df['forecast_label']=forecast_df['forecast_label'].astype('string')
        # print(forecast_df.dtypes)

        return forecast_df